In [1]:
from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
query1 = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
query2 = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
O9DataLake.register("sales_df",DataSource.LS, ResourceType.IBPL, query1) 
O9DataLake.register("DPBaseInputs",DataSource.LS, ResourceType.IBPL, query2)

2023-01-30 06:02:37,801 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/ExecuteCompactJsonQuery]
2023-01-30 06:02:37,805 : srishtisingh_5502 :Starting new HTTPS connection (1): myazureqatest.o9solutions.com:443
2023-01-30 06:02:37,999 : srishtisingh_5502 :https://myazureqatest.o9solutions.com:443 "POST /api/ibplquery/5502/ExecuteCompactJsonQuery HTTP/1.1" 200 None
2023-01-30 06:02:43,008 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/ExecuteCompactJsonQuery]
2023-01-30 06:02:43,012 : srishtisingh_5502 :Starting new HTTPS connection (1): myazureqatest.o9solutions.com:443
2023-01-30 06:02:43,193 : srishtisingh_5502 :https://myazureqatest.o9solutions.com:443 "POST /api/ibplquery/5502/ExecuteCompactJsonQuery HTTP/1.1" 200 None
2023-01-30 06:02:48,296 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/Ex

,Version.[Version Name],Product.[Product],Time.[FiscalWeek],SalesAccount.[Account],Location.[Location],DPSellOutUnitsActuals,Mean Pricing Save PCT,Placement Count,Promotion Count,DPSellOutPrice
0,CurrentWorkingView,196426,W03-2016,ALL,ALL,11.0,NaN,NaN,NaN,10.0
1,CurrentWorkingView,196426,W05-2016,ALL,ALL,11.0,NaN,NaN,NaN,10.0
2,CurrentWorkingView,196426,W08-2016,ALL,ALL,11.0,NaN,NaN,NaN,10.0
3,CurrentWorkingView,196426,W13-2016,ALL,ALL,11.0,NaN,NaN,NaN,10.0
4,CurrentWorkingView,196426,W48-2016,ALL,ALL,11.0,NaN,NaN,NaN,10.0
...,...,...,...,...,...,...,...,...,...,...
162,CurrentWorkingView,208821,W48-2018,ALL,ALL,NaN,0.5,1.0,1.0,NaN
163,CurrentWorkingView,208821,W49-2018,ALL,ALL,NaN,NaN,1.0,NaN,NaN
164,CurrentWorkingView,208821,W50-2018,ALL,ALL,NaN,NaN,1.0,NaN,NaN
165,CurrentWorkingView,208821,W51-2018,ALL,ALL,NaN,NaN,1.0,NaN,NaN


In [2]:
"""
exec plugin instance [CsvMGIP1_CsvMGIP2_CsvMGPrimDf_CsvMGOP1_Slice] for measures {[DPSellOutUnitsActuals]} using scope ([Product].[Product].Filter(#.Name in {[196426],[208821]}) * [Time].[Day] * [Version].[Version Name].[CurrentWorkingView]);

"""
import logging
from sklearn import tree

logger = logging.getLogger('o9_logger')

out_df = None
input_df = O9DataLake.get('sales_df')
predict_df = O9DataLake.get('DPBaseInputs')
if predict_df is not None and len(predict_df.index) > 0:
    x = input_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']]
    y = input_df[['DPSellOutUnitsActuals']]
    x = x.fillna(0).values
    y = y.fillna(0).values

    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(x, y)

    predict = predict_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']].fillna(0).values
    # predict_keys = predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]',                               'Location.[Location]']]
    # predict_keys["Product.[Product]"] = pd.to_numeric(predict_keys["Product.[Product]"], downcast='integer')
    predict_values = clf.predict(predict)
    predict_df["DPSellOutUnitsFcst"] = predict_values
    #out_df = pd.concat([predict_keys, out_df_part], axis=1)
    out_df=predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]','Location.[Location]','DPSellOutUnitsFcst']]
    logger.info(out_df)

else:
    out_df = None

O9DataLake.put('DPBaseOutput1',out_df)

2023-01-30 06:03:08,277 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/ExecuteCompactJsonQuery]
2023-01-30 06:03:08,279 : srishtisingh_5502 :Starting new HTTPS connection (1): myazureqatest.o9solutions.com:443
2023-01-30 06:03:08,359 : srishtisingh_5502 :https://myazureqatest.o9solutions.com:443 "POST /api/ibplquery/5502/ExecuteCompactJsonQuery HTTP/1.1" 200 None
2023-01-30 06:03:13,368 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/ExecuteCompactJsonQuery]
2023-01-30 06:03:13,371 : srishtisingh_5502 :Starting new HTTPS connection (1): myazureqatest.o9solutions.com:443
2023-01-30 06:03:13,466 : srishtisingh_5502 :https://myazureqatest.o9solutions.com:443 "POST /api/ibplquery/5502/ExecuteCompactJsonQuery HTTP/1.1" 200 None
2023-01-30 06:03:18,491 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/Ex